### **Install Dependencies**

In [ ]:
!pip install transformers datasets

In [ ]:
pip install transformers[torch]

## **GPT2 Model**

### **Intialize the tokenizer and load the pre trained model**

In [ ]:


# Load the dataset
dataset = load_dataset("heliosbrahma/mental_health_chatbot_dataset")
texts = dataset['train']['text']

# Load GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set the pad_token_id for the tokenizer
tokenizer.pad_token_id = tokenizer.eos_token_id

# Tokenize the conversations
tokenized_conversations = [torch.tensor(tokenizer.encode(text)) for text in texts]
# tokenized_conversations = [tokenizer.encode(text) for text in texts]

# Create a custom dataset
from torch.utils.data import Dataset

class ChatDataset(Dataset):
    def __init__(self, conversations):
        self.conversations = conversations

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        conversation = self.conversations[idx]
        input_ids = conversation[:-1]
        target_ids = conversation[1:]
        return input_ids, target_ids

dataset = ChatDataset(tokenized_conversations)

### **Load the dataset and Train the model**

In [ ]:
# Set up the training parameters
batch_size = 8
learning_rate = 5e-5
num_epochs = 100

# Create a custom collation function
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids, target_ids = zip(*batch)
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    target_ids = pad_sequence(target_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    return input_ids, target_ids

# Create a data loader
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Set up the optimizer and learning rate scheduler
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * num_epochs)

# Fine-tune the model
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs, labels=targets)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        progress_bar.set_postfix(loss=loss.item())
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/100, Loss: 4.128930612043901


Epoch 2/100, Loss: 3.202729181809859


Epoch 3/100, Loss: 2.861420848152854


Epoch 4/100, Loss: 2.7911785299127754


...

Epoch 96/100, Loss: 0.28201956234195014


Epoch 97/100, Loss: 0.27529002793810586


Epoch 98/100, Loss: 0.2662598571994088


Epoch 99/100, Loss: 0.27739235216921027


Epoch 100/100, Loss: 0.27072104202075437


In [ ]:
torch.save(model, '/content/drive/MyDrive/chatbot_models/gpt_2_100e.pth')

In [ ]:
model = torch.load('/content/drive/MyDrive/chatbot_models/gpt_2_100e.pth')


### **Chatbot**

In [ ]:
# Function to chat with the bot
def chat_with_bot():
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Chatbot: Goodbye!")
            break
        # Tokenize user input
        user_input_ids = tokenizer.encode(user_input, return_tensors='pt')

        user_input_ids = user_input_ids.to(device)
        # Generate response
        bot_output = model.generate(user_input_ids, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
        # Decode and print response
        bot_response = tokenizer.decode(bot_output[0], skip_special_tokens=True)
        print("Chatbot:", bot_response)

# Start chatting
chat_with_bot()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Evaluation**

In [ ]:
!pip install scikit-learn

In [ ]:
import torch
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.nn.utils.rnn import pad_sequence

# Load the dataset
dataset = load_dataset("heliosbrahma/mental_health_chatbot_dataset")
texts = dataset['train']['text']

# Load the trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set the pad_token_id for the tokenizer
tokenizer.pad_token_id = tokenizer.eos_token_id

# Tokenize the conversations
tokenized_conversations = [torch.tensor(tokenizer.encode(text)) for text in texts]

# Create a custom dataset
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, conversations):
        self.conversations = conversations

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        conversation = self.conversations[idx]
        input_ids = conversation[:-1]
        target_ids = conversation[1:]
        return input_ids, target_ids

dataset = ChatDataset(tokenized_conversations)

# Create a data loader with custom collation function
def collate_fn(batch):
    input_ids, target_ids = zip(*batch)
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    target_ids = pad_sequence(target_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    return input_ids, target_ids

batch_size = 8
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Evaluate the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

total_loss = 0
total_accuracy = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs, labels=targets)
        loss = outputs.loss
        total_loss += loss.item()

        # Calculate accuracy
        predicted_ids = torch.argmax(outputs.logits, dim=-1)
        accuracy = (predicted_ids == targets).float().mean().item()
        total_accuracy += accuracy

avg_loss = total_loss / len(test_loader)
avg_accuracy = total_accuracy / len(test_loader)

print(f"Average Loss: {avg_loss:.4f}")
print(f"Average Accuracy: {avg_accuracy:.4f}")


Average Loss: 0.0785
Average Accuracy: 0.4056


In [ ]:
import torch.nn.functional as F

def calculate_perplexity(model, test_loader, device):
    total_perplexity = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs, labels=targets)
            log_probs = F.log_softmax(outputs.logits, dim=-1)
            perplexity = torch.exp(-torch.mean(log_probs.gather(2, targets.unsqueeze(2)).squeeze(2)))
            total_perplexity += perplexity.item()
    avg_perplexity = total_perplexity / len(test_loader)
    return avg_perplexity

avg_perplexity = calculate_perplexity(model, test_loader, device)
print(f"Average Perplexity: {avg_perplexity:.4f}")


Average Perplexity: 687.9785
